<a href="https://colab.research.google.com/github/Ethan-Jeong/test_deeplearning/blob/master/NLP/topic_modeling_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from https://wikidocs.net/30707

In [1]:
!curl -O https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 51.5M  100 51.5M    0     0   103M      0 --:--:-- --:--:-- --:--:--  103M


In [2]:
import pandas as pd

In [3]:
df_data = pd.read_csv('./abcnews-date-text.csv')
df_data.head(3)

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit


In [4]:
df_data = df_data.head(10000)

In [5]:
head_text = df_data[['headline_text']]
type(head_text)

pandas.core.frame.DataFrame

In [6]:
import nltk

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
head_text['title_text'] = head_text.apply(lambda row:nltk.word_tokenize(row['headline_text']),axis=1)
head_text.head(3)


,headline_text,title_text
0,aba decides against community broadcasting lic...,"[aba, decides, against, community, broadcastin..."
1,act fire witnesses must be aware of defamation,"[act, fire, witnesses, must, be, aware, of, de..."
2,a g calls for infrastructure protection summit,"[a, g, calls, for, infrastructure, protection,..."


In [9]:
from nltk.corpus import stopwords

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [21]:
head_text['title']  = head_text['title_text'].apply(lambda x:[ word for word in x if ( len(word) > 3 ) if ( word not in stop ) ])

In [22]:
head_text.head(5)

,headline_text,title_text,title
0,aba decides against community broadcasting lic...,"[aba, decides, against, community, broadcastin...","[decides, community, broadcasting, licence]"
1,act fire witnesses must be aware of defamation,"[act, fire, witnesses, must, be, aware, of, de...","[fire, witnesses, must, aware, defamation]"
2,a g calls for infrastructure protection summit,"[a, g, calls, for, infrastructure, protection,...","[calls, infrastructure, protection, summit]"
3,air nz staff in aust strike for pay rise,"[air, nz, staff, in, aust, strike, for, pay, r...","[staff, aust, strike, rise]"
4,air nz strike to affect australian travellers,"[air, nz, strike, to, affect, australian, trav...","[strike, affect, australian, travellers]"


In [23]:
head_text['title'][3]

['staff', 'aust', 'strike', 'rise']

In [24]:
tokens = []
for i in  range(len(head_text)):
  tokens.append(' '.join(head_text['title'][i]))

tokens[3:5]


['staff aust strike rise', 'strike affect australian travellers']

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
tfidf = TfidfVectorizer(max_features=1000)
X = tfidf.fit_transform(tokens)

In [27]:
X.shape

(10000, 1000)

In [ ]:
X[4].toarray()

In [29]:
from sklearn.decomposition import LatentDirichletAllocation

In [32]:
lda_model = LatentDirichletAllocation(max_iter=10) # max_iter 반복 횟수

In [33]:
lda_top = lda_model.fit_transform(X)

In [43]:
lda_model.components_.shape , lda_model.components_ # 중요도

((10, 1000), array([[ 0.10000501,  0.10000536,  0.10000945, ...,  0.10000033,
          0.10000144,  0.10000411],
        [ 1.08875447,  0.10000217,  0.10000355, ...,  0.10000346,
         12.4138707 ,  0.10000187],
        [ 0.10000192,  0.1       ,  0.10000092, ...,  0.10000204,
          0.10000279,  0.10000226],
        ...,
        [ 8.80138048,  0.10000158,  0.1       , ...,  0.10000263,
          0.10000376, 14.75125348],
        [ 0.10001038,  0.1000053 ,  0.10001143, ...,  0.10000365,
          0.10000578,  0.10000354],
        [ 0.10000233,  1.23924541,  0.10000285, ..., 18.59647596,
          0.10000947,  0.10001654]]))

In [45]:
terms = tfidf.get_feature_names()

In [46]:
n = 5
for idx, topic in enumerate(lda_model.components_):
  print([(terms[i], topic[i]) for i in topic.argsort()[:-n-1:-1]])


[('final', 37.19687594850525), ('dies', 35.91478217469809), ('charged', 34.31420532529283), ('claim', 33.52769740614824), ('claims', 31.483509746658836)]
[('council', 59.66502286060179), ('police', 39.35460830821631), ('plans', 30.889469085332106), ('centre', 29.14601611224007), ('funds', 29.04420140652721)]
[('iraq', 79.65638622557462), ('aust', 40.19943060271765), ('troops', 38.757483183101265), ('clash', 36.22265776816468), ('title', 33.6622997906459)]
[('plan', 40.10649784427741), ('found', 34.4223426954813), ('group', 29.937120798418437), ('move', 27.301120870440915), ('says', 27.06166292109257)]
[('hospital', 52.426506441858265), ('water', 47.42397442636168), ('take', 30.18779233328432), ('still', 30.09507056147541), ('work', 29.603480295217256)]
[('anti', 41.19262564998499), ('protesters', 34.34772577394548), ('community', 32.15659141871279), ('continues', 31.827650730131435), ('bush', 31.195181547151343)]
[('coast', 39.30287382233011), ('urged', 35.20518961577955), ('open', 32.